# timeseries
* 時系列データの前処理について書いていく。

# csvの読み込み
* csvの読み込みにはpd.read_csvを用いることが多いと思う
* オプションの `parse_dates` を用いると特定の行を `datetime`形式として取り込める

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

## サンプルのcsvファイルを作成

In [2]:
start_time = datetime(2017, 11, 1, 10, 0, 0, 0)
time_list = []
for i in range(100):
    t = start_time + timedelta(seconds=i)
    time_list.append(t)
    
price = 50000
price_list = []
for _ in range(100):
    price += np.random.randint(-500,500)
    price_list.append(price)
    

df = pd.DataFrame([time_list,price_list]).T
df.columns=["date_time","price"]
df.head()

,date_time,price
0,2017-11-01 10:00:00,49859
1,2017-11-01 10:00:01,49874
2,2017-11-01 10:00:02,49757
3,2017-11-01 10:00:03,49495
4,2017-11-01 10:00:04,49430


In [3]:
df.to_csv("tp_sample.csv",index=False)

## parse_dates を用いない場合

In [4]:
df_no_parse = pd.read_csv("tp_sample.csv")
df_no_parse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
date_time    100 non-null object
price        100 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


In [5]:
type(df_no_parse.date_time[0])

str

In [6]:
from datetime import timedelta
df_no_parse.date_time[0] + timedelta(seconds=1)

TypeError: must be str, not datetime.timedelta

* `parse_dates` を用いないと、 `str` として扱われる
* `timedelta` を用いた時間の足し算等もできない

## parse_dates を用いる場合
* `read_csv` のオプション `parse_dates` にリストで、 `datetime` として扱いたいカラム名を与えてあげる。

In [7]:
df_parse = pd.read_csv("tp_sample.csv",parse_dates=["date_time"])
df_parse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
date_time    100 non-null datetime64[ns]
price        100 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.6 KB


In [8]:
type(df_parse.date_time[0])

pandas._libs.tslib.Timestamp

In [9]:
df_parse.date_time[0]

Timestamp('2017-11-01 10:00:00')

In [10]:
df_parse.date_time[0] + timedelta(seconds=1)

Timestamp('2017-11-01 10:00:01')

* `datetime` として扱われ、 `timedelta` を用いた時間の足し算もできている。

# 差分
* 時系列データは1時点離れたデータとの差を用いることが多い。このようなデータを差分系列、階差系列と呼ぶ。

### サンプルデータの作成

In [11]:
# 時間のリストの効率的な作り方がわからない
start_time = datetime(2017, 11, 1, 10, 0, 0, 0)
time_list = []
for i in range(100):
    t = start_time + timedelta(seconds=i)
    time_list.append(t)
    
price = 50000
price_list = []
for _ in range(100):
    price += np.random.randint(-500,500)
    price_list.append(price)

In [12]:
df = pd.DataFrame(price_list,index=time_list,columns=["price"])
df.head(10)

,price
2017-11-01 10:00:00,49501
2017-11-01 10:00:01,49239
2017-11-01 10:00:02,49666
2017-11-01 10:00:03,50036
2017-11-01 10:00:04,50067
2017-11-01 10:00:05,49913
2017-11-01 10:00:06,50003
2017-11-01 10:00:07,49540
2017-11-01 10:00:08,49956
2017-11-01 10:00:09,49579


* 上記のような秒間データがあったとする。

### 差分の作成

* pd.DataFrame.diff()を用いることで、差分を取ることができる。

In [13]:
df["diff_1"] = df.diff()
df.head()

,price,diff_1
2017-11-01 10:00:00,49501,NaN
2017-11-01 10:00:01,49239,-262.0
2017-11-01 10:00:02,49666,427.0
2017-11-01 10:00:03,50036,370.0
2017-11-01 10:00:04,50067,31.0


* periodsを指定することで、差分を取る間隔を変更することができる。

In [14]:
df["diff_3"] = df["price"].diff(periods=3)
df.head()

,price,diff_1,diff_3
2017-11-01 10:00:00,49501,NaN,NaN
2017-11-01 10:00:01,49239,-262.0,NaN
2017-11-01 10:00:02,49666,427.0,NaN
2017-11-01 10:00:03,50036,370.0,535.0
2017-11-01 10:00:04,50067,31.0,828.0


# 移動平均

## rolling
* rollingを用いると、移動平均のような、DataFrameを上からなぞりつつ、特定の行数分だけ着目し、計算するということができる。
* 下記は、3秒ごとの平均（移動平均）を求めている。
* `window` で秒数を指定することができるのは便利。
* なお、下記のように、`window` に時間を用いている場合は、オプションの `center` を用いることはできない。

In [15]:
df["rolling_mean_3s"] = df["price"].rolling(window="3s").mean()
df.head()

,price,diff_1,diff_3,rolling_mean_3s
2017-11-01 10:00:00,49501,NaN,NaN,49501.000000
2017-11-01 10:00:01,49239,-262.0,NaN,49370.000000
2017-11-01 10:00:02,49666,427.0,NaN,49468.666667
2017-11-01 10:00:03,50036,370.0,535.0,49647.000000
2017-11-01 10:00:04,50067,31.0,828.0,49923.000000


* 上記は、ある行から過去に遡って3秒の行の平均を求めている。
* `2017-11-01 10:00:02` の行であれば、 `2017-11-01 10:00:00`, `2017-11-01 10:00:01`, `2017-11-01 10:00:02` の行の `price` を足し、平均し、 `2017-11-01 10:00:02` の行の `rolling_mean_3s` の列に記載している。
* `2017-11-01 10:00:00`はその前2秒のデータがないためあるだけの（その行だけの）平均を求めている。